In [1]:
!pip install --upgrade pip

  Using cached pip-25.1-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-25.1-py3-none-any.whl (1.8 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\ABDULLAH\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
!pip install -Uq openai-agents


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [21]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

api_key = os.getenv("GEMINI_API_KEY")
MODEL = "gemini-2.0-flash"

In [22]:
from pyexpat import model
from agents import Agent , Runner , AsyncOpenAI , OpenAIChatCompletionsModel , function_tool
from agents.run import RunConfig

# Check if the API key is present; if not, raise an error
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [25]:
from importlib.abc import InspectLoader


@function_tool
def get_current_weather(city:str):
    """Get Current Weather Informations"""
    print("--- weather tool Called ---")
    return f"The Weather is Sunny in {city}"

weather_agent = Agent(
    name="weather_agent",
    instructions="You are an Expert weather agent answer the user query for weather.",
    model=model,
    tools=[get_current_weather]
)

result = Runner.run_sync(weather_agent , "What is The Current weather in Karachi" , run_config=config)
print(result.final_output)

--- weather tool Called ---
The Weather is Sunny in Karachi


# Real Time Weather Update With Tool Calling

In [30]:
import requests
@function_tool
def get_weather(city:str)->str:
    print(f"--- Current Weather Tool Called for {city} --- ")
    """
    Get the current weather for a given city.
    """
    result=requests.get(f"http://api.weatherapi.com/v1/current.json?key=8e3aca2b91dc4342a1162608252604&q={city}")
    data=result.json()
    return f"The current weather in {city} is {data['current']['temp_c']}°C with {data['current']['condition']['text']}."


In [32]:
agent = Agent(
    name="weather_agent",
    instructions="You are an Expert weather agent answer the user query for weather.",
    model=model,
    tools=[get_weather]
)

result = Runner.run_sync(agent , "What is The Current weather in Bombay" , run_config=config)
print(result.final_output)

--- Current Weather Tool Called for Bombay --- 
The current weather in Bombay is 32.4°C with Mist.



In [41]:
@function_tool
def getWeather(city: str) -> str:
    """
    Get the weather for a given city.
    """
    # Replace with your actual weather API URL and key
    result = requests.get(f"http://api.weatherapi.com/v1/current.json?key=8e3aca2b91dc4342a1162608252604&q={city}")
    
    if result.status_code == 200:
        data = result.json()
        return f"The weather in {city} is {data['current']['temp_c']}°C with {data['current']['condition']['text']}."
    else:
        return "Sorry, I couldn't fetch the weather data."

In [42]:
weather_agent1 = Agent(
    name="weather_agent",
    instructions="You are an Expert weather agent answer the user query for weather.",
    model=model,
    tools=[getWeather]
)

In [43]:
result = Runner.run_sync(weather_agent1 , "What is The Current weather in Bombay" , run_config=config)
print(result.final_output)

The weather in Bombay is 32.4°C with Mist.

